In [1]:
import rasterio as rs
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString

In [2]:
d = {'geometry': [Polygon([(458452, 6034982),(458552,6034982), (458552, 6035042), (458452, 6035042)]),
                 Polygon([(458452, 6034982),(458552,6034982), (458552, 6035042), (458452, 6035042)])]}
gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")

In [ ]:
# This can bue used to get the data in the defined batches and save them seperately in the different tif files

for i in range(2):
    minx,miny,maxx,maxy = gdf.loc[i,'geometry'].bounds
    height = int(np.floor((maxx - minx)/0.5))
    width = int(np.floor((maxy - miny)/0.5))
    xx = np.arange(minx,maxx,0.5)
    yy = np.arange(miny,maxy,0.5)
    XX,YY = np.meshgrid(xx + (0.5//2), yy +(0.5//2))
    XY = np.vstack([XX.ravel(),YY.ravel()])
    rowcol = np.asarray(img.index(XX.ravel(),YY.ravel())).T
    df = pd.DataFrame()
    for b in img.indexes:
        band = img.read(b,out_dtype='float64')
        extracted_values = [band[coord[0], coord[1]] for coord in rowcol]
        df['CHM_'+str(b)] = extracted_values
    mat = df.values.reshape(width,height,img.count)
    transform = rs.transform.from_bounds(minx,miny,maxx,maxy, width, height)
    file_name = 'test{}.tif'.format(i)
    with rs.open(file_name, "w",
                   driver='GTiff',
                   count=img.count,
                   transform = transform,
                   width=width,
                   height=height,
                   dtype='float64',
                   crs="epsg:3346",) as output_file:
        output_file.write(np.moveaxis(mat[::-1],-1,0))
    
    print(minx,miny,maxx,maxy)